In [1]:
import ujson
import pprint
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from operator import itemgetter
import multiprocessing

pp = pprint.PrettyPrinter(indent=4)
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Testing auto adding time output to all cells
'''
#To install...
wget https://raw.githubusercontent.com/cpcloud/ipython-autotime/master/autotime.py
#Make available via
jupyter nbextension install /Users/patrickmulrooney/class/notebooks/pjmulroo/DSECapstone/autotime.py
'''
 
%load_ext autotime

In [3]:
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

# for _line in log_progress(lines,every=1):
#     _line = _line.strip()

time: 41.7 ms


In [4]:
!ls /Users/patrickmulrooney/Downloads/

PIT_commands.pdf    endomondo.json.gz   endomondoHR.json.gz
endomondo           endomondoHR
time: 156 ms


# Overview

-- Data pulled from 'http://jmcauley.ucsd.edu/data/endomondo/endomondo.json.gz' and 'http://jmcauley.ucsd.edu/data/endomondo/endomondoHR.json.gz' on 2016.11.26.

-- MD5 of uncompressed files at time or download:

```
753c9b9afdb9e27076b831985c52bf33  endomondo.json
800036c402949822a71cdf69143c56d3  endomondoHR.json
```


# Parse endomondo.json

Following commands used to get data in proper format:


```
cd ~/Downloads/
# Replace all the single quotes with double, valid json
perl -pe "s/\'/\"/g" endomondo.json > endomondo_parsed1.json
# Add comma to end of each line, make valid json
perl -pe "s/$/,/" endomondo_parsed1.json > endomondo_parsed2.json 
# Remove temp file
rm endomondo_parsed1.json
# Split file into more managable files
mkdir endomondo
cd endomondo
split -l 100000 ../endomondo_parsed2.json endomondo_parsed
# Append [ to beginning of file, and replace last ',' with closing bracket
# to make files valid JSON
for i in `ls`; do sed "1s/^/\[/;$ s/.$/]/" $i > $i.json;done
```

In [5]:
## Figure out later
# # Debugging version, time: 29min 24s
# endoFiles = glob.glob("/Users/patrickmulrooney/Downloads/endomondo/*json")

# endo = []

# for _file in log_progress(endoFiles):

#     with open(_file, 'r') as f:
#         # Just testing the import, runs out of memory
#         # TODO: Figure out why or how to deal with.
# #         endo += ujson.load(f)
#         endo.append(ujson.load(f))


time: 2.55 ms


# endomondoHR.json
## Parse

Following commands used to get data in proper format:


```
cd ~/Downloads/
# Replace all the single quotes with double, valid json
perl -pe "s/\'/\"/g" endomondoHR.json > endomondoHR_parsed1.json
# Add comma to end of each line, make valid json
perl -pe "s/$/,/" endomondoHR_parsed1.json > endomondoHR_parsed2.json 
# Remove temp file
rm endomondoHR_parsed1.json
# Split file into more managable files
mkdir endomondoHR
cd endomondoHR
split -l 100000 ../endomondoHR_parsed2.json endomondoHR_parsed
# Append [ to beginning of file, and replace last ',' with closing bracket
# to make files valid JSON
for i in `ls`; do sed "1s/^/\[/;$ s/.$/]/" $i > $i.json;done
```

In [6]:
!gzip -vd /Users/patrickmulrooney/Downloads/endomondoHR/endomondoHR_parseda*json.gz

gzip: can't stat: /Users/patrickmulrooney/Downloads/endomondoHR/endomondoHR_parseda*json.gz (/Users/patrickmulrooney/Downloads/endomondoHR/endomondoHR_parseda*json.gz.gz): No such file or directory
time: 150 ms


In [7]:
# time: 4min 4s
endoHRFiles = glob.glob("/Users/patrickmulrooney/Downloads/endomondoHR/*json")

endoHR = []

for _file in log_progress(endoHRFiles, every=1):

    with open(_file, 'r') as f:
        endoHR += ujson.load(f)


time: 3min 47s


In [8]:
print "Got %s records"%len(endoHR)

Got 253020 records
time: 13 ms


### Looks like only two variations of the data exist as far as series. Those that include the speed and those that don't.

```
{   (u'altitude', u'longitude', u'heart_rate', u'timestamp', u'latitude'): 203359,
    (u'altitude', u'longitude', u'heart_rate', u'timestamp', u'latitude', u'speed'): 49659}
```

In [9]:
## Not as good as cell below, use it instead.

# # time: 55min 4s

# # Go through each HR record and find out which keys it contains whose values are a list.
# # All list values should be part of a time series which we should combine into a single key value.

# endoHRListTypes = ['timestamp', 'heart_rate', 'altitude', 'latitude', 'longitude', 'speed']

# seriesLength = []

# endoHR2 = []
# for _endoHR in log_progress(endoHR, every=1):

#     # New dict to store parsed values
#     _endoHRtmp = {}

#     # Iterate through keys and find those not of type list, we deal with those below
#     for j in _endoHR.keys():
#         if type(_endoHR[j]) != list:
#             _endoHRtmp[j] = _endoHR[j]

#     # For debugging purposes, get a record of length of series for histogram
#     _seriesLen = len(_endoHR['timestamp'])
#     seriesLength.append(_seriesLen)

#     # Combine the list types for each record into one.
#     # Probably should do more sanity checking of length of lists
#     # If data not timestamped pretty useless
#     _endoHRtmp['series'] = []
#     for j in xrange(_seriesLen):
#         # Speed is the only field missing, so deal with it
#         try:
#             _endoHRtmp['speed_included'] = True
#             _endoHRtmp['series'].append( [ _endoHR['timestamp'][j], _endoHR['heart_rate'][j], _endoHR['altitude'][j], \
#                                            _endoHR['latitude'][j], _endoHR['longitude'][j], _endoHR['speed'][j] ] )
#         except:
#             _endoHRtmp['speed_included'] = False
#             _endoHRtmp['series'].append( [ _endoHR['timestamp'][j], _endoHR['heart_rate'][j], _endoHR['altitude'][j], \
#                                            _endoHR['latitude'][j], _endoHR['longitude'][j] ] )            

#     endoHR2.append(_endoHRtmp)

time: 6.92 ms


In [10]:
## Not as good as cell below, use it instead.

# # time: 54min 45s
# # Version2 of cell above, inplace instead of new
# # Go through each HR record and find out which keys it contains whose values are a list.
# # All list values should be part of a time series which we should combine into a single key value.

# endoHRListTypes = ['timestamp', 'heart_rate', 'altitude', 'latitude', 'longitude', 'speed']

# seriesLength = []

# for _endoHR in log_progress(endoHR, every=1):

#     # For debugging purposes, get a record of length of series for histogram
#     _seriesLen = len(_endoHR['timestamp'])
#     seriesLength.append(_seriesLen)

#     # Combine the list types for each record into one.
#     # Probably should do more sanity checking of length of lists
#     # If data not timestamped pretty useless
#     _endoHR['series'] = []
#     for j in xrange(_seriesLen):
#         # Speed is the only field missing, so deal with it
#         try:
#             _endoHR['speed_included'] = True
#             _endoHR['series'].append( [ _endoHR['timestamp'][j], _endoHR['heart_rate'][j], _endoHR['altitude'][j], \
#                                            _endoHR['latitude'][j], _endoHR['longitude'][j], _endoHR['speed'][j] ] )

#         except:
#             _endoHR['speed_included'] = False
#             _endoHR['series'].append( [ _endoHR['timestamp'][j], _endoHR['heart_rate'][j], _endoHR['altitude'][j], \
#                                            _endoHR['latitude'][j], _endoHR['longitude'][j] ] )

#     # Remove unused keys
#     _endoHR.pop('timestamp', None)
#     _endoHR.pop('heart_rate', None)
#     _endoHR.pop('altitude', None)
#     _endoHR.pop('latitude', None)
#     _endoHR.pop('longitude', None)
#     if u'speed' in _endoHR.keys():
#         _endoHR.pop('speed', None)


time: 54min 45s


In [10]:
# time: 41min 15s
# Version3 of cell above, inplace instead of new, using del instead of pop.
# Go through each HR record and find out which keys it contains whose values are a list.
# All list values should be part of a time series which we should combine into a single key value.
# Not sure why it is so slow, should be faster.

endoHRListTypes = ['timestamp', 'heart_rate', 'altitude', 'latitude', 'longitude', 'speed']

seriesLength = []

for _endoHR in log_progress(endoHR, every=5):

    # For debugging purposes, get a record of length of series for histogram
    _seriesLen = len(_endoHR['timestamp'])
    seriesLength.append(_seriesLen)

    # Combine the list types for each record into one.
    # Probably should do more sanity checking of length of lists
    # If data not timestamped pretty useless
    _endoHR['series'] = []
    for j in xrange(_seriesLen):
        # Speed is the only field missing, so deal with it
        try:
            _endoHR['speed_included'] = True
            _endoHR['series'].append( [ _endoHR['timestamp'][j], _endoHR['heart_rate'][j], _endoHR['altitude'][j], \
                                           _endoHR['latitude'][j], _endoHR['longitude'][j], _endoHR['speed'][j] ] )

        except:
            _endoHR['speed_included'] = False
            _endoHR['series'].append( [ _endoHR['timestamp'][j], _endoHR['heart_rate'][j], _endoHR['altitude'][j], \
                                           _endoHR['latitude'][j], _endoHR['longitude'][j] ] )

    # Remove unused keys
    del _endoHR['timestamp']
    del _endoHR['heart_rate']
    del _endoHR['altitude']
    del _endoHR['latitude']
    del _endoHR['longitude']
    if u'speed' in _endoHR.keys():
        del _endoHR['speed']


time: 41min 15s


## Write parsed data to avoid needing to run this multiple times

In [12]:
# time: 3min 41s

with open('/Users/patrickmulrooney/Downloads/endomondoHR/endoHRParsed.json', 'w') as f:
     ujson.dump(endoHR, f)

time: 3min 41s


## Move on to notebook 2
Link to [ExploreData - 2](ExploreData - 2.ipynb)